In [1]:
from MA_utils import *
from MA_utils.AudioConnection import AudioConn
from MA_utils.audio_data_augumentation import AudioDataAugumentation

AC = AudioConn()
AudioDA = AudioDataAugumentation()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
INPUT_DIR = os.path.realpath(AC.DATA_DIR) + '\\'
OUTPUT_DIR = os.path.realpath(os.path.join('.','data_DA_output')) + '\\'
#if not os.path.exists(OUTPUT_DIR): os.mkdir(OUTPUT_DIR)

In [3]:
print(INPUT_DIR)
print(OUTPUT_DIR)

E:\ML_Learning\Music_analysis\data\
E:\ML_Learning\Music_analysis\data_DA_output\


In [4]:
def clone_input_dir_structure(INPUT_DIR, OUTPUT_DIR):
    for dirpath, dirnames, filenames in os.walk(INPUT_DIR):
        structure = os.path.join(OUTPUT_DIR, dirpath[len(INPUT_DIR):])
        if not os.path.isdir(structure):
            os.mkdir(structure)
        else:
            print(f"{structure} \n This folder does already exits! \n ----------------------------------------------- \n")

In [9]:
clone_input_dir_structure(INPUT_DIR, OUTPUT_DIR)

In [10]:
def create_new_data_from_raw_input(INPUT_DIR, OUTPUT_DIR):
    for dirpath, dirnames, filenames in os.walk(INPUT_DIR):
            for filename in filenames: 
                if '.wav' in filename:
                    new_subdir = ''
                    output_path = os.path.join(OUTPUT_DIR, dirpath[len(INPUT_DIR):], new_subdir, filename)
                    AudioDA.transpose_wav_file(path_input = os.path.join(dirpath, filename),
                                               path_output = output_path,
                                               n_steps=0)
                    

In [11]:
create_new_data_from_raw_input(INPUT_DIR, OUTPUT_DIR)

In [13]:
def generate_transpose_matrix(labels):
    result_mtx = pd.DataFrame(labels)
    for idx, label in enumerate(labels):
        labels.append(labels.pop(0))
        result_mtx[idx+1] = pd.Series(labels)
    return result_mtx

In [12]:
if not os.path.exists(os.path.join(OUTPUT_DIR, 'labels')):
    os.mkdir(os.path.join(OUTPUT_DIR, 'labels'))

for label in ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']:
    if not os.path.exists(os.path.join(OUTPUT_DIR, 'labels', label)):
        os.mkdir(os.path.join(OUTPUT_DIR, 'labels', label))

Transponujemy nagrania do odpowiednich tonacji

In [28]:
labels = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
transpose_matrix = generate_transpose_matrix(labels)

for dirpath, dirnames, filenames in os.walk(os.path.join(OUTPUT_DIR, 'chords')):
        for filename in filenames:
            
            filename_tonation = filename.split('.')[0]
            transpose_row = transpose_matrix[transpose_matrix[0] == filename_tonation]
            
            for step in transpose_row.columns:
                output_path = os.path.join(OUTPUT_DIR, 'labels', transpose_row[step].values[0], dirpath.split('\\')[-1] + '_from_{}'.format(filename_tonation) + '.wav' )
                AudioDA.transpose_wav_file(path_input = os.path.join(dirpath, filename),
                                           path_output= output_path, 
                                           n_steps=step)                

Generujemy dane w różnych tempach

In [33]:
for dirpath, dirnames, filenames in os.walk(os.path.join(OUTPUT_DIR, 'labels')):
    for filename in filenames:
        
        filename_wo_wav = filename.split('.')[0]
                
        for time_rate in np.linspace(0.75, 1.25, 11):
            
            output_path = os.path.join(dirpath, filename_wo_wav + '_{}'.format(str(time_rate).replace('.','_')) + '.wav')
            AudioDA.chtime_wav_file(path_input = os.path.join(dirpath, filename),
                                    path_output= output_path,
                                    rate = time_rate)                